In [1]:
import spacy
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
import log_analyzer as lo_an
import solrhandler as sh
import clusterer as cls
import topicdeterminator as td
from service_clustering import ServiceClustering

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#query = "anmeldung"
#target_service = 120686

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
#eval = eval.head(3)
eval.head(20)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...


In [4]:
#eval.iloc[2]["initialQuestion"]

Funktionen

In [5]:
def findCorrectAnswer(chatbot,targetService):
    
    # Find Target Cluster
    clusteredColumn = "cluster_label"
    service = chatbot.df_current.loc[chatbot.df_current["id"]==str(targetService)]
    if 'cluster_label' not in service.columns:
        return None
    
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]
    
    if targetCluster == chatbot.max_cluster_label_current:
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df_current.reset_index()
    try:
        rank = temp.index[temp["id"]==str(targetService)].values[0]
    except IndexError as e:
        return float("NaN")
    return rank + 1

Logging

In [6]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None, initialQuery = False): 
        target_service = row["documentId"]
        #print(ID)
        #print(t)
        #print(answer)
       
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        if initialQuery==True:
            nResults = len(self.chatbot.df_current.index)
            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, dialogId, ID, t, service_name, query, "initialQuery", None, rank, nResults)
        else:
            if answer is not None:

                
                nResults = len(self.chatbot.df_current.index)
                question = self.chatbot.generateQuestion()
                answer = answer

                rank = getRank(self.chatbot,target_service)# getRank(target_service)

                row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)

            else:
                row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

TEST

Execution

In [7]:
kmeans = KMeans(n_clusters=2)
#chatbot= ServiceClustering(query, kmeans, max_results=3)

In [8]:
# Initialisierung Komponenten Chatbot
#solrhandler = sh.SolrHandler(1000)
#clusterer = cls.Clusterer()
#tpc_dterminator = td.TopicDeterminator()

from sklearn.cluster import DBSCAN


In [9]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    print(ID)
    query = row["initialQuestion"]
    target_service = row["documentId"]
    t = 0
    
    # Chatbot initialisieren
    try: # 1.3
        chatbot =  ServiceClustering(query, kmeans, max_results=1)
    
    # Fehler überspringen: eg. kein solr output
    except Exception as e: 
        if(str(e) == 'response'): # TODO ??
            print(e)
            next
        elif(str(e) == 'no solr output'):
            print(e)
            next
        elif(str(e) == 'cluster_label'):
            print(e)
            next    
        else:
            print(e)
            next
            
            
    lg.setChatbot(chatbot)
    
    
    answer = findCorrectAnswer(chatbot, target_service)
    
    if answer is None: # service nicht in resultset, überspringen
        next
    lg.log(ID,t, row, answer, initialQuery = True)
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):
        
        # log
        t += 1
        
#         print(ID)
#         print(t)
#         print(np.unique(chatbot.df["ssdsLemma_cluster"]))
#         print("")
        
        chatbot.refineResultset(answer)
        answer = findCorrectAnswer(chatbot, target_service)
        
        if answer is None: 
            break
        lg.log(ID, t, row,answer)
        
        #print(chatbot.clusterer.clustering_algorithm.eps)

0
1
2


In [10]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "name", "query", "question", "answer", "rank", "nResult"])
log_df.to_csv("service_clus_kmeans_n1.csv")

Log Analyzer

In [11]:
log_analyzer=lo_an.Log_Analyzer(log_df)
results=log_analyzer.run_it()

In [12]:
results[3]

{'Mean Turns': 2.3333333333333335,
 '90-percentile Turns': 4.4,
 '75% quantile Turns': 3.5,
 '25% quantile Turns': 1.0,
 'Mean nResults Reduct': 0.44827315541601254,
 'Harmonic mean nResult Reduct': 0.39622641509433965,
 'Mean delta nResults': 2.5714285714285716,
 'Mean delta Rank': 0.7142857142857143,
 'Median Turns': 2.0,
 'Max Turns needed': 5,
 'Mean information gain': 0.9296849485998138}

Statistics ( Alt)

In [13]:
# avg Rank Improvement in turn t
#log_df["delta_Rank"] = log_df.groupby(["ID"])["rank"].diff()
#log_df.groupby(["t"])["rank_improvement"].mean()

In [14]:
# avg nSuggestions Improvement in turn t
#log_df["delta_nResults"] = log_df.groupby(["ID"])["nResults"].diff()
#log_df.groupby(["t"])["sugg_improvement"].mean()

In [15]:
# MRR ohne cluster refinement
#df = log_df.loc[log_df["t"] == 1]
#MRR_ohne = 1/(df["rank"].mean())
#MRR_ohne

In [16]:
# MRR mit cluster refinement
#df = log_df.groupby(["ID"]).last("t")
#MRR_mit = 1/(df["rank"].mean())
#MRR_mit
#df.head()

In [17]:
# avg Turns
#print(log_df["t"].mean())

# bei durchschnittlich nResults
#print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

In [18]:
#log_df.head(20)